In [ ]:
!pip install --upgrade google-cloud-storage


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-storage-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.92.0 requires google-cloud-storage<3.0.0,>=1.32.0, but you have google-cloud-storage 3.1.0 which is incompatible.


In [ ]:
!pip install --upgrade google-cloud-storage


Defining Project ID from Google Cloud bucket and selecting the Region

In [ ]:
PROJECT_ID="lyrical-rampart-459603-a8"
REGION="us-central1"

Authenticating Google Cloud login and signup

In [ ]:
! gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=O3DjPCpbWPeb8gKugg0SlBlWjvYMo3&prompt=consent&token_usage=remote&access_type=offline&code_challenge=dfMmwLBcRDHDWkvciKsQTDCsIE6iExPPylOFOvPfyKA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AUJR-x7pqKn3j9ihwz8ZkB6mQyjm-2yO7A32dk2QTVpgFhtGb9v8VuVYtncGvKxKXeDPIQ

You are now logged in as [rohithgollapalli2000@gmail.com].
Your current

Configuring Google Cloud and updating project

In [ ]:
! gcloud config set project "lyrical-rampart-459603-a8"

Updated property [core/project].


Authenticating Google colab with google cloud

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
#BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"

Creating New Bucket on Google Cloud

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

CommandException: "mb" command does not support "file://" URLs. Did you mean to use a gs:// URL?


Importing files form local drive eg CSV format and converting it into JSONL format and upload it to gcs bucket

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

filename = next(iter(uploaded))
df = pd.read_csv(filename)
df.head()

jsonl_str  = ""

# Create a string for JSONL format
for i, row in df.iterrows():
    #print(row['Inline_text'], row['Label'])
    desc = row['Inline_text'].replace("'",'').replace('"',"").replace("\n","")
    jsonl_str += '''{"systemInstruction": {"role": "system","parts": [{"text": "You are a Classifier responsible to classify into CTS,CTS Campus,Classroom Support,Print Team (External)."}]}, "contents": [{"role": "user","parts": [{"text": 'DESC'}]},{"role": "model","parts": [{"text": 'LABEL'}]}]}'''.replace("DESC", desc).replace("LABEL", row['Label'])
    jsonl_str += "\n"

#with open("sample_data/updated_email.jsonl", 'w') as file:
    #file.write(jsonl_str)
output_path = "sample_data/updated_email.jsonl"
with open(output_path, 'w') as file:
    file.write(jsonl_str)
bucket_name = "your-bucket-name-lyrical-rampart-459603-a8-unique"
!gcloud storage cp {output_path} gs://{bucket_name}/

Saving emaildescprtion.csv to emaildescprtion.csv
Copying file://sample_data/updated_email.jsonl to gs://your-bucket-name-lyrical-rampart-459603-a8-unique/updated_email.jsonl


Fintuning AI model with JSONL file

In [ ]:

import time

import vertexai
from vertexai.tuning import sft

# TODO(developer): Update and un-comment below line
#PROJECT_ID = "lyrical-rampart-459603-a8"
vertexai.init(project=PROJECT_ID, location="us-central1")

sft_tuning_job = sft.train(
    source_model="gemini-2.0-flash-001",
    # 1.5 and 2.0 models use the same JSONL format
    train_dataset="gs://your-bucket-name-lyrical-rampart-459603-a8-unique/updated_email.jsonl",
)

# Polling for job completion
while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(f'model name: {sft_tuning_job.tuned_model_name}')
print(f'endpoint: {sft_tuning_job.tuned_model_endpoint_name}')
print(f'experiment: {sft_tuning_job.experiment}')
print(f'tuning job: {sft_tuning_job}')
# Example response:
# projects/123456789012/locations/us-central1/models/1234567890@1
# projects/123456789012/locations/us-central1/endpoints/123456789012345
# <google.cloud.aiplatform.metadata.experiment_resources.Experiment object at 0x7b5b4ae07af0>

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob
INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/3231630305152466944?project=69210017139


model name: projects/69210017139/locations/us-central1/models/1739237179630026752@1
endpoint: projects/69210017139/locations/us-central1/endpoints/8848013060783210496
experiment: <google.cloud.aiplatform.metadata.experiment_resources.Experiment object at 0x7b29e5d1e3d0>
tuning job: <vertexai.tuning._supervised_tuning.SupervisedTuningJob object at 0x7b29e5d1e450> 
resource name: projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944


In [ ]:
!gcloud ai hp-tuning-jobs list --region=us-central1



Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Listed 0 items.


View a list of tuning jobs

In [ ]:
import vertexai
from vertexai.tuning import sft

# TODO(developer): Update and un-comment below line
# PROJECT_ID = "your-project-id"
vertexai.init(project=PROJECT_ID, location="us-central1")

responses = sft.SupervisedTuningJob.list()

for response in responses:
    print(response)
# Example response:
# <vertexai.tuning._supervised_tuning.SupervisedTuningJob object at 0x7c85287b2680>
# resource name: projects/12345678/locations/us-central1/tuningJobs/123456789012345

resource name: projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944
resource name: projects/69210017139/locations/us-central1/tuningJobs/7223697540862967808
resource name: projects/69210017139/locations/us-central1/tuningJobs/6083723885184811008
resource name: projects/69210017139/locations/us-central1/tuningJobs/7989731689981018112
resource name: projects/69210017139/locations/us-central1/tuningJobs/975885443746758656


Get details of a tuning job

In [ ]:
import vertexai
from vertexai.tuning import sft

# TODO(developer): Update and un-comment below lines
#PROJECT_ID = "lyrical-rampart-459603-a8"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

tuning_job_id = "7223697540862967808"
response = sft.SupervisedTuningJob(f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}")

print(response)
# Example response:
# <vertexai.tuning._supervised_tuning.SupervisedTuningJob object at 0x7cc4bb20baf0>
# resource name: projects/1234567890/locations/us-central1/tuningJobs/4982013113894174720

resource name: projects/69210017139/locations/us-central1/tuningJobs/7223697540862967808


In [ ]:
from vertexai.generative_models import GenerativeModel
#PROJECT_ID = "lyrical-rampart-459603-a8"
LOCATION = "us-central1"
tuning_job_id = "7223697540862967808"


sft_tuning_job = sft.SupervisedTuningJob(f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}")
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)
content = "I have an ipad to reset password."
print(tuned_model.generate_content(content))

candidates {
  content {
    role: "model"
    parts {
      text: "Reset Password"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.2915094792842865
}
usage_metadata {
  prompt_token_count: 8
  candidates_token_count: 2
  total_token_count: 10
  prompt_tokens_details {
    modality: TEXT
    token_count: 8
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 2
  }
}
create_time {
  seconds: 1747182114
  nanos: 246990000
}
response_id: "IuIjaM6JD4mr1dkP5PW56QY"



Testing AI model with Prompts

In [ ]:
from vertexai.generative_models import GenerativeModel

sft_tuning_job = sft.SupervisedTuningJob(f"projects/{PROJECT_ID}/locations/{REGION}/tuningJobs/{tuning_job_id}")
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)

content = '''
{
  "systemInstruction": {
    "role": "system",
    "parts": [
        {
            "text": "You are a Classifier responsible to classify into CTS,CTS Campus,Classroom Support,Print Team (External)."
        }
    ]
  },
  {
    "contents": [
        {
            "role": "USER",
            "parts": {
                "text" : "Projector in mcnair is not working."
            },
  {
    "contents": [
        {
            "role": "USER",
            "parts": {
                "text" : ""
            }
        },
    ],
    "generation_config": {
        "temperature":0.2
    }
}
'''

print(tuned_model.generate_content(content))

candidates {
  content {
    role: "model"
    parts {
      text: "Classroom Support"
    }
  }
  finish_reason: STOP
  avg_logprobs: -1.0105397701263428
}
usage_metadata {
  prompt_token_count: 172
  candidates_token_count: 2
  total_token_count: 174
  prompt_tokens_details {
    modality: TEXT
    token_count: 172
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 2
  }
}
create_time {
  seconds: 1747182116
  nanos: 105198000
}
response_id: "JOIjaO61Br6o1dkPioDXiAM"

